In [ ]:
import pandas as pd
from math import *
import numpy as np

df = pd.read_csv('uber.csv')
df
df.info()
df.describe()
df = df.dropna()
df.duplicated().sum()
df = df.drop(columns=['Unnamed: 0','key','pickup_datetime'])
df = df[df['fare_amount']!=0]
df

# To calculate distance between pickup and dropoff
def cal_dis(df):
    x1 = df['pickup_latitude']
    y1 = df['pickup_longitude']
    x2 = df['dropoff_latitude']
    y2 = df['dropoff_longitude']
    
    r = 6378.137
    lat1, lon1, lat2, lon2 = map(radians, [x1, y1, x2, y2])  # Convert degree to radian
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = r * c
    return distance

df['distance'] = df.apply(cal_dis, axis=1)
df
df['distance'].describe()
df = df[df['distance']!=0]

import matplotlib.pyplot as plt
import seaborn as sns

sns.boxplot(df['distance'])
df[df['distance']>10000]
df = df[df['distance']<10000]
sns.boxplot(df['fare_amount'])
df[df['fare_amount']>400]
df = df[df['fare_amount']<400]
sns.boxplot(df['passenger_count'])
df = df[df['passenger_count']<150]
sns.boxplot(df['passenger_count'])
df[df['passenger_count']>5.5]
df.corr()
sns.heatmap(df.corr(), annot=True)

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
df['distance'] = sc.fit_transform(df[['distance']])
df['passenger_count'] = sc.fit_transform(df[['passenger_count']])
df

x = df[['distance', 'passenger_count']]
y = df['fare_amount']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
pred = linear_model.predict(x_test)

from sklearn.metrics import r2_score, mean_squared_error
print("R2 score is : ", r2_score(y_test, pred))
print("RMSE score is : ", np.sqrt(mean_squared_error(y_test, pred)))

rfe = RandomForestRegressor()
rfe.fit(x_train, y_train)
pred = rfe.predict(x_test)
print("R2 score is : ", r2_score(y_test, pred))
print("RMSE score is : ", np.sqrt(mean_squared_error(y_test, pred)))
